In [32]:
import os
import math
import random
from pprint import pprint

#### 0. Configuration
*(Modify here only)*

In [33]:
params = {
  "H_NUM_OF_ROWS" : 300,
  "H_NUM_OF_COLS" : 21,         # WARNING: Do not set as powers of 2.

  "W_NUM_OF_ROWS" : 21,
  "W_NUM_OF_COLS" : 16,

  "NUM_OF_NODES"  : 18,          # WARNING: Do not set as powers of 2.
}

In [34]:
CORA_DATASET_ENABLE = 1         # 1: enable for Cora dataset    0: enable for other dataset
UPDATE_INPUT_TXT    = 1         # 1: update inputs/*.txt        0: do not update inputs/*.txt

#### 1. Random Inputs

In [35]:
params.update({
  "H_DATA_DEPTH"    : 0,                                # số lượng col_idx tổng cộng
  "NODE_INFO_DEPTH" : 0,                                # số lượng node tổng cộng
  "WEIGHT_DEPTH"    : 0,                                # = W_ROWS * W_COLS
  "WH_DEPTH"        : 0,                                # = số lượng node tổng cộng
  "A_DEPTH"         : params["W_NUM_OF_COLS"] * 2,      # = W_COLS * 2
})

In [36]:
def random_matrix(rows, cols, sparsity_percent, min=-127, max=127):
  neg_percent = 20
  total_elements = rows * cols
  num_zeros = int(total_elements * sparsity_percent / 100)
  num_non_zeros = total_elements - num_zeros

  num_negatives = int(num_non_zeros * neg_percent / 100)
  num_positives = num_non_zeros - num_negatives

  matrix_elements = (
      [0] * num_zeros +
      [random.randint(min, -1) for _ in range(num_negatives)] +
      [random.randint(1, max) for _ in range(num_positives)]
  )

  random.shuffle(matrix_elements)

  sparse_matrix = []
  for i in range(rows):
    sparse_matrix.append(matrix_elements[i * cols:(i + 1) * cols])

  # Ensure no row is completely zero
  for row in sparse_matrix:
    if all(value == 0 for value in row):
      row[random.randint(0, cols - 1)] = random.randint(1, 9)

  return sparse_matrix

##### Weight


In [37]:
weight = random_matrix(params["W_NUM_OF_ROWS"], params["W_NUM_OF_COLS"], 0, -31, 31)

##### Weight Vector a

In [38]:
# a = [1 if i < params["A_DEPTH"] / 2 else 2 for i in range(params["A_DEPTH"])]
a = random_matrix(1, params["A_DEPTH"], 0)[0]

midpoint = len(a) // 2
a_1 = a[:midpoint]
a_2 = a[midpoint:]

##### SubGraph

In [39]:
subGraph = []
sub_row = random.randint(2, params["NUM_OF_NODES"])
sub_cnt = 0
source_node_list = []
while sub_cnt < params["H_NUM_OF_ROWS"]:
  sub_row = random.randint(2, params["NUM_OF_NODES"])
  sub_cnt += sub_row
  if sub_cnt >= params["H_NUM_OF_ROWS"]:
    sub_row = sub_row - (sub_cnt - params["H_NUM_OF_ROWS"])
    if sub_row == 1:
      sub_row = 2
      for item in subGraph:
        if len(item) >= 3:
          item.pop(-1)
          break
  subG = random_matrix(sub_row, params["H_NUM_OF_COLS"], 50, -15, 15)
  if (CORA_DATASET_ENABLE):
    subG = [[1 if x != 0 else 0 for x in row] for row in subG]
  subGraph.append(subG)

##### Feature

In [40]:
feature = []
for i in range(len(subGraph)):
  feature += subGraph[i]

#### 2. Calculation

##### SPMM

In [41]:
wh = [[0 for _ in range(len(weight[0]))] for _ in range(len(feature))]
for i in range(len(feature)):
  for j in range(len(weight[0])):
    for k in range(len(weight)):
      if (feature[i][k] != 0):
        wh[i][j] += feature[i][k] * weight[k][j]

def matrix_multiplication(subgraph, weight_matrix):
  rows = len(subgraph)
  cols = len(weight_matrix[0])
  common_dim = len(weight_matrix)

  result_matrix = [[0 for _ in range(cols)] for _ in range(rows)]

  for i in range(rows):
    for j in range(cols):
      for k in range(common_dim):
        result_matrix[i][j] += subgraph[i][k] * weight_matrix[k][j]
  return result_matrix

WH = []
for i in range(len(subGraph)):
	wh = matrix_multiplication(subGraph[i], weight)
	WH.append(wh)

##### DMVM

In [42]:
DMVM = []
COEF = []

for wh in WH:
  src = 0
  dmvm_subgraph = []
  coef_subgraph = []
  for idx, row in enumerate(wh):
    dst       = 0
    node_row  = []
    a_in      = a_1 if idx == 0 else a_2

    # multiply
    for i in range(len(row)):
      node_row.append(row[i] * a_in[i])

    # add
    product_len = len(row)
    while product_len > 1:
      for j in range(len(row)):
        if (2*j+1) < product_len:
          node_row[j] = node_row[2*j] + node_row[2*j+1]
      product_len /= 2
    dst = node_row[0]

    # final
    if (idx == 0):
      src = dst
    dmvm_subgraph.append(dst)

    if (src + dst < 0):
      coef_subgraph.append(0)
    else:
      if ((src + dst) // pow(2, 12) > 0):
        coef_subgraph.append((src + dst) // pow(2, 12))
      else:
        coef_subgraph.append(0)
  DMVM.append(dmvm_subgraph)
  COEF.append(coef_subgraph)

##### Softmax

In [43]:
def binary_of_2_power_x(x):
  if not (0 <= x <= 127):
    raise ValueError("Input x must be in the range 0 to 127")
  result = 2 ** x
  binary_representation = bin(result)[2:]  # Convert to binary and strip '0b'
  binary_representation = binary_representation.zfill(103)  # Pad to 103 bits

  return binary_representation

In [44]:
SOFTMAX = []
DIVIDEND = []
DIVISOR = []
SM_NUM_NODE = []
EXP_ALPHA = []
for subgraph in COEF:
  softmax = []
  sum = 0
  exp = []
  sum_exp = 0

  for ele in subgraph:
    softmax.append(pow(2, ele))
    sum += pow(2, ele)
    exp.append(math.exp(ele))
    sum_exp += math.exp(ele)

  for i in range(len(softmax)):
    DIVIDEND.append(softmax[i])
    softmax[i] = softmax[i] / sum
    exp[i] = exp[i] / sum_exp

  DIVISOR.append(sum)
  SM_NUM_NODE.append(len(softmax))
  SOFTMAX.append(softmax)
  EXP_ALPHA.append(exp)

##### Aggregator

In [45]:
# TODO:

#### 3. GCSR structure

In [46]:
source_node_list = []
num_nodes_subgraph = []
node_info = []
col_idx = []
value = []
non_zero_ele = []

row_length_bits = math.ceil(math.log2(params["H_NUM_OF_COLS"]))
num_of_nodes_bits = math.ceil(math.log2(params["NUM_OF_NODES"]))

for i in range(len(subGraph)):
  source_node = subGraph[i][0]
  num_node = len(subGraph[i])
  source_node_list.append(source_node)
  num_nodes_subgraph.append(num_node)
  node_info_list = []
  for j in range(len(subGraph[i])):
    if j == 0:
      flag = '1'
    else:
      flag = '0'
    non_zero_values = [ele for k, ele in enumerate(subGraph[i][j]) if ele != 0]
    # print("Non Zero Values = ", non_zero_values)
    row_length = len(non_zero_values)
    node_info_bin = (f"{row_length:0{row_length_bits}b}"f"{num_node:0{num_of_nodes_bits}b}" f"{flag}")
    node_info_list.append(node_info_bin)

  node_info.append(node_info_list)

def decode_node_info(node_info_string):
  # Extract each part from the string
  flag_bits = 1
  row_length = int(node_info_string[:row_length_bits], 2)
  num_of_nodes = int(node_info_string[row_length_bits:row_length_bits+num_of_nodes_bits], 2)

  flag = node_info_string[-flag_bits:]  # '1' for source node, '0' for neighbor node

  # Return a dictionary for better readability
  return {
    'row_length': row_length,
    'num_of_nodes': num_of_nodes,
    'is_source_node': flag == '1'
  }

# Example usage
decoded_node_info = []
for nd_info in node_info:
  decoded_info = [decode_node_info(info) for info in nd_info]
  decoded_node_info.append(decoded_info)
# pprint(decoded_node_info)

In [47]:
def extract_from_raw(raw_list):
	col_idx = []
	value = []
	node_info = []
	for row in raw_list:
		non_zero_elements = [(j, ele) for j, ele in enumerate(row) if ele != 0]
		col_idx.extend([idx for idx, _ in non_zero_elements])
		value.extend([val for _, val in non_zero_elements])
		row_len = len(non_zero_elements)

	return col_idx, value

#### 4. SubGraph

In [48]:
GRAPH = {}
idx = 0

for i in range(len(subGraph)):
  col_idx_, value_ = extract_from_raw(subGraph[i])
  GRAPH[i] = {"raw" : subGraph[i], "col_idx" : col_idx_, "value" : value_, "node_info" : node_info[i], "WH": WH[i],  "DMVM": DMVM[i], "Coef": COEF[i], "Softmax": SOFTMAX[i]}

In [49]:
colors = {
  "raw": "\033[91m",          # Red
  "col_idx": "\033[92m",      # Green
  "value": "\033[93m",        # Yellow
  "node_info": "\033[94m",    # Blue
  "WH": "\033[95m",           # Magenta
  "DMVM": "\033[96m",         # Cyan
  "Coef": "\033[97m",         # White
  "Softmax": "\033[1;91m",      # Bright Black (Gray)
  "Aggregator": "\033[1;94m", # Bright Blue (a lighter shade of blue)
  "reset": "\033[0m"          # Reset color
}

def print_sub_graphs():
  for subgraph in GRAPH.values():
    print("-" * 90)
    for i in range(len(subgraph["raw"])):
      if i == 0:
        print(f"{colors['raw']}raw\t\t{subgraph['raw'][i]}{colors['reset']}")
      else:
        print(f"{colors['raw']}\t\t{subgraph['raw'][i]}{colors['reset']}")

    print(f"{colors['col_idx']}col_idx\t\t{subgraph['col_idx']}{colors['reset']}")
    print(f"{colors['value']}value\t\t{subgraph['value']}{colors['reset']}")
    print(f"{colors['node_info']}node_info\t{subgraph['node_info']}{colors['reset']}")

    for i in range(len(subgraph["WH"])):
      if i == 0:
        print(f"{colors['WH']}SPMM\t\t{subgraph['WH'][i]}{colors['reset']}")
      else:
        print(f"{colors['WH']}\t\t{subgraph['WH'][i]}{colors['reset']}")

    print(f"{colors['DMVM']}DMVM\t\t{[subgraph['DMVM'][idx] for idx in range(len(subgraph['raw']))]}{colors['reset']}")
    print(f"{colors['Coef']}COEF\t\t{[subgraph['Coef'][idx] for idx in range(len(subgraph['raw']))]}{colors['reset']}")
    print(f"{colors['Softmax']}SOFTMAX\t\t{[subgraph['Softmax'][idx] for idx in range(len(subgraph['raw']))]}{colors['reset']}")
    print("-" * 90)

#### 5. SubGraph Information

In [50]:
print_sub_graphs()

------------------------------------------------------------------------------------------
raw		[0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
		[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
		[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0]
		[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0]
		[1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]
		[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]
		[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1]
		[1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0]
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0]
		[1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0]
		[1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
		[1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
		[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1]
		[1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 

#### 6. Update testbench

In [51]:
root_path         = os.path.abspath("../../").replace("\\", "/")
tb_path           = os.path.join(root_path, "tb").replace("\\", "/")
filename          = os.path.join(root_path, "tb/top_tb.sv").replace("\\", "/")
file_path_input   = os.path.join(root_path, "tb/input").replace("\\", "/")
file_path_output  = os.path.join(root_path, "tb/output")
params_pkg_file   = os.path.join(root_path, "rtl/others/pkgs/params_pkg.sv").replace("\\", "/")

In [52]:
print(tb_path)

d:/VLSI/Capstone/tb


In [53]:
def decimal_to_binary(number, bit_length):
  if number < 0:
    number = (1 << bit_length) + number  # Handle negative numbers with two's complement
  binary_representation = bin(number & ((1 << bit_length) - 1))[2:]  # Mask to bit_length bits
  return binary_representation.zfill(bit_length)

In [54]:
wrt_data = []
wrt_node_info = []

for key, value in GRAPH.items():
  wrt_node_info.append(value["node_info"])

  data_bin = []
  for i in range(len(value["value"])):
    col_idx_bin = decimal_to_binary(value["col_idx"][i], math.ceil(math.log2(params["H_NUM_OF_COLS"])))
    value_bin = decimal_to_binary(value["value"][i], 8)
    data_bin.append(col_idx_bin + value_bin)
  wrt_data.append(data_bin)

In [55]:
def flatten_list(nested_list):
  flattened = []
  for item in nested_list:
    if isinstance(item, list):
      flattened.extend(flatten_list(item))
    else:
      flattened.append(item)
  return flattened

def generate_input(file_path, data_list):
  data = flatten_list(data_list)
  with open(file_path, 'w') as file:
    print(f" open {file_path}" )
    for item in data:
      file.write(f"{item}\n")

In [56]:
def decimal_to_binary(value, num_bits, signed=False):
    if signed:
        min_val = -(2 ** (num_bits - 1))
        max_val = (2 ** (num_bits - 1)) - 1
        if not (min_val <= value <= max_val):
            raise ValueError(f"Value {value} cannot be represented in {num_bits} signed bits.")
        if value < 0:
            value = (1 << num_bits) + value  # Two's complement
    else:
        if not (0 <= value < (1 << num_bits)):
            raise ValueError(f"Value {value} cannot be represented in {num_bits} unsigned bits.")
    return f"{value:0{num_bits}b}"

In [57]:
def dec_to_fxp(decimal_num, is_signed, int_bits, frac_bits):
  if is_signed and decimal_num < 0:
    total_bits = int_bits + frac_bits
    decimal_num += (1 << total_bits)
  scaled_num = round(decimal_num * (1 << frac_bits))
  binary_rep = f'{scaled_num:0{int_bits + frac_bits}b}'
  return binary_rep

def generate_golden_output(file_path, data_list):
  data = flatten_list(data_list)
  with open(file_path, 'w') as file:
    print(f" open {file_path}" )
    for item in data:
      file.write(f"{item}\n")

##### Inputs & Outputs

In [58]:
if (UPDATE_INPUT_TXT):
  generate_input(file_path_input + "/h_data.txt", wrt_data)
  generate_input(file_path_input + "/node_info.txt", wrt_node_info)
  generate_input(file_path_input + "/weight.txt", weight)
  generate_input(file_path_input + "/a.txt", a)

generate_golden_output(file_path_output + "/SPMM/wh.txt", WH)
generate_golden_output(file_path_output + "/DMVM/dmvm.txt", DMVM)
generate_golden_output(file_path_output + "/DMVM/coef.txt", COEF)
generate_golden_output(file_path_output + "/softmax/alpha.txt", SOFTMAX)
generate_golden_output(file_path_output + "/softmax/divisor.txt", DIVISOR)
generate_golden_output(file_path_output + "/softmax/dividend.txt", DIVIDEND)
generate_golden_output(file_path_output + "/softmax/num_nodes.txt", SM_NUM_NODE)
generate_golden_output(file_path_output + "/softmax/exp_alpha.txt", EXP_ALPHA)

 open d:/VLSI/Capstone/tb/input/h_data.txt
 open d:/VLSI/Capstone/tb/input/node_info.txt
 open d:/VLSI/Capstone/tb/input/weight.txt
 open d:/VLSI/Capstone/tb/input/a.txt
 open d:/VLSI/Capstone\tb/output/SPMM/wh.txt
 open d:/VLSI/Capstone\tb/output/DMVM/dmvm.txt
 open d:/VLSI/Capstone\tb/output/DMVM/coef.txt
 open d:/VLSI/Capstone\tb/output/softmax/alpha.txt
 open d:/VLSI/Capstone\tb/output/softmax/divisor.txt
 open d:/VLSI/Capstone\tb/output/softmax/dividend.txt
 open d:/VLSI/Capstone\tb/output/softmax/num_nodes.txt
 open d:/VLSI/Capstone\tb/output/softmax/exp_alpha.txt


In [59]:
_value = [len(item) for item in wrt_data]
len_value = 0
for val in _value:
  len_value += val

_node_info = [len(item) for item in wrt_node_info]
len_node_info = 0
for node in _node_info:
  len_node_info += node

params["H_DATA_DEPTH"] = len_value
params["NODE_INFO_DEPTH"] = len_node_info
params["WEIGHT_DEPTH"] = params["W_NUM_OF_ROWS"] * params["W_NUM_OF_COLS"]
params["WH_DEPTH"] = len_node_info
params["A_DEPTH"] = params["W_NUM_OF_COLS"] * 2

In [60]:
config_params = {
  "H_NUM_SPARSE_DATA" : params["H_DATA_DEPTH"],
  "TOTAL_NODES"       : params["H_NUM_OF_ROWS"],
  "NUM_FEATURE_IN"    : params["W_NUM_OF_ROWS"],
  "NUM_FEATURE_OUT"   : params["W_NUM_OF_COLS"],
  "NUM_SUBGRAPHS"     : len(GRAPH),
  "MAX_NODES"         : params["NUM_OF_NODES"],
}

##### Parameters

In [61]:
def update_parameters_in_ifdef(params_pkg_file, parameters, ifdef_condition="SIMULATION"):
  with open(params_pkg_file, 'r') as file:
    file_data = file.readlines()

  in_ifdef_block = False  # Track if we're inside the target `ifdef` block
  for i, line in enumerate(file_data):
    # Check for the start of the `ifdef block
    if f"`ifdef {ifdef_condition}" in line:
      in_ifdef_block = True
      continue

    # Check for the end of the `ifdef block
    if in_ifdef_block and "`else" in line:
      in_ifdef_block = False
      continue

    # Update parameters only within the `ifdef block
    if in_ifdef_block:
      for param, new_value in parameters.items():
        found_param = f"parameter {param}"
        if found_param in line:
          parts = line.split('=')
          if len(parts) == 2:  # Ensure the line has an '=' sign
            file_data[i] = f"{parts[0]}= {new_value};\n"

  # Write back the updated content
  with open(params_pkg_file, 'w') as file:
    file.writelines(file_data)

# Example usage
if UPDATE_INPUT_TXT:
  update_parameters_in_ifdef(params_pkg_file, config_params)

##### Root Path

In [62]:
def update_root_path(filename, root_path):
  with open(filename, 'r') as file:
    file_data = file.readlines()

  for i, line in enumerate(file_data):
    if "localparam string ROOT_PATH" in line:
      file_data[i] = f"\tlocalparam string ROOT_PATH = \"{root_path}\";\n"

  with open(filename, 'w') as file:
      file.writelines(file_data)

update_root_path(filename, tb_path)